In [1]:
import re
import pandas as pd
import numpy as np

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# Load dataset
data = pd.read_csv('../preprocessed_congressional_tweet.csv')

data.head(10)

,favorite_count,length,COVID19,SOTU,tcot,Obamacare,ForThePeople,coronavirus,ACA,SCOTUS,...,NC13,Yes2Energy,ma09,AUMF,VA5,MN03,OH16,Other,retweet_count,party_id
0,0.000000,0.028556,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000047,R
1,0.000604,0.065386,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000520,R
2,0.000000,0.029090,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000009,R
3,0.000021,0.022151,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000014,R
4,0.000007,0.061649,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000014,D
5,0.000033,0.046704,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000061,D
6,0.000009,0.030424,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000014,R
7,0.000000,0.020550,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,R
8,0.000009,0.016013,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000042,R
9,0.000012,0.011209,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.000000,D


In [3]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode labels from string (neg or pos) to integer (0 or 1)
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['party_id'])

# One-hot encoding
Y = to_categorical(integer_encoded) 

In [4]:
Y[0]

array([0., 1.], dtype=float32)

In [5]:
data_array = data.values

X_train, X_test, Y_train, Y_test = train_test_split(data_array[:,:-1], Y, test_size=0.25, random_state=42)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print(X_train.shape)

(444602, 504)


In [6]:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

2022-05-08 14:34:21.728585: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               258560    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 390,402
Trainable params: 390,402
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Import callbacks from keras
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

file_path = '/tmp/checkpoint'  # for ModelCheckpoint callback that needs file path to call

# Initialize the callbacks and add them to a list
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_on_plateau = ReduceLROnPlateau(monitor="loss", mode="min", factor=0.1, patience=5, verbose=1)
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

callbacks_list = [checkpoint, reduce_on_plateau, es]

In [8]:
history = model.fit(X_train, Y_train, batch_size=512, epochs=20, callbacks=callbacks_list, verbose=1, 
                    validation_data=(X_test, Y_test))

Epoch 1/20
866/869 [============================>.] - ETA: 0s - loss: 0.5005 - accuracy: 0.7084
Epoch 1: val_accuracy improved from -inf to 0.71505, saving model to /tmp/checkpoint


2022-05-08 14:34:31.810354: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
869/869 [==============================] - 10s 11ms/step - loss: 0.5005 - accuracy: 0.7084 - val_loss: 0.4879 - val_accuracy: 0.7150 - lr: 0.0010
Epoch 2/20
864/869 [============================>.] - ETA: 0s - loss: 0.4885 - accuracy: 0.7159
Epoch 2: val_accuracy improved from 0.71505 to 0.71613, saving model to /tmp/checkpoint
INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
869/869 [==============================] - 8s 9ms/step - loss: 0.4885 - accuracy: 0.7160 - val_loss: 0.4882 - val_accuracy: 0.7161 - lr: 0.0010
Epoch 3/20
863/869 [============================>.] - ETA: 0s - loss: 0.4877 - accuracy: 0.7170
Epoch 3: val_accuracy improved from 0.71613 to 0.71833, saving model to /tmp/checkpoint
INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
869/869 [==============================] - 8s 9ms/step - loss: 0.4877 - accuracy: 0.7169 - val_loss: 0.4862 - val_accuracy: 0.7183 - lr: 0.0010
Epoch 4/20
865/869 [====

In [9]:
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

4632/4632 [==============================] - 5s 1ms/step - loss: 0.4845 - accuracy: 0.7206
Test loss: 0.48453524708747864
Test accuracy: 0.7205889225006104


In [10]:
i = 3
predict_array = model.predict(X_test[[i],:])

# Index of the max value (the name of the class with the highest probability)
prediction = np.argmax(predict_array)
truth = np.argmax(Y_test[i])

print(f"Prediction: {prediction}, Truth: {truth}")

Prediction: 0, Truth: 0
